We compute in this worksheet the orbifold fundamental group $G_1$ of Proposition 4.3 in the paper `Torsion divisors of plane curves with maximal flexes and Zariski pairs` by E. Artal, S. Bannai, T. Shirane and H. Tokunaga. As for other groups we use the package `sirocco` by M.Á Marco and M. Rodríguez. Since we need to know which are the meridians of the irreducible components (including the line at infinity), instead of using the method `fundamental_group` we use `braid_monodromy`.

We introduce the curve. The equations are in a number field; $f$ is the cubic, $T_1$ is the equation of one triangle (which contains the line at infinity) and $T_2$ is the equation of another triangle (this polynomial has three linear factors in an extension of the number field).

In [1]:
R0.<t0>=QQ[]
p0=t0^2 - 3*t0 + 9
a0=p0.roots(QQbar)[0][0]
L.<u0>=NumberField(p0,embedding=a0)
S1.<x,y>=L[]
f=x^2*y+y^2+x
T1=x*y
T2=x^3 + y^3 + u0*x*y + 1
F=(f*T1*T2)(x=x+y)

In [2]:
%%time
C=Curve(F)
L=C.braid_monodromy()

CPU times: user 1.54 s, sys: 1.4 s, total: 2.94 s
Wall time: 1min 43s


In [3]:
len(L)

14

The braid monodromy has $14$ braids (one of the them corresponding to two points where the curve is non transversal to the vertical line). We define functions to simplify the words of the braids and to separate the obvious conjugating braids.

In [4]:
def trconm(L):
    A=[]
    for j in range(len(L)):
        if j not in A:
            L1=[k for k in range(j+1,len(L)) if L[j]+L[k]==0]
            if L1!=[]:
                k=L1[0]
                L2=[i for i in L[j+1:k] if abs(abs(i)-abs(L[j]))<2]
                if L2==[]:
                    A+=[j,k]
    return tuple(L[j] for j in range(len(L)) if j not in A)

def trconm2(L0):
    L1=[_ for _ in L0]
    L2=trconm(L1)
    while len(L2)<len(L1):
        L1=[_ for _ in L2]
        L2=trconm(L1)
    return(L2)

def romper(L0):
    c0=()
    b0=L0
    while b0[0]+b0[-1]==0:
        c0=c0+(b0[0],)
        b0=b0[1:-1]
    return (c0,b0)

In [5]:
F1=FreeGroup(8)
B=BraidGroup(8)

In [6]:
pares0=[]
for aux in L:
    aux=B(F1(aux.Tietze()).Tietze())
    aux=B(trconm2(aux.Tietze()))
    pares0.append(romper(aux.Tietze()))

We want to modify the above decomposition in order to have positive braids for the middle braids. The result will be kept in the list `puiseux`. Each element of the list `puiseux` have four entries: two braids $\tau_1,\tau_2$ ($\tau_2$ is a positive algebraic braid) and two numbers $m,n$. The braid associated to the basis element is $\tau_1\cdot\tau_2\cdot\tau_1^{-1}$; $m$ is the first strand involved in $\tau_2$ and $n$ is the number of strands involved in $\tau_2$.

In order to keep these computations as short as possible, we need to manipulate the braids one by one.

In [7]:
puiseux=[]

In [8]:
print(pares0[0])

((1, 4, 5, 6, 4, 5, 3, 4, -3, 5, 2, 3, 4, 7, 6, 7, 1, 5, 2, 4, 6, 3, 4, 5, 7, 6, 7, 4, -6, -7), (6, 5, 6, 5, 5, 6, 5, 6))


The first braid has the following decomposition, which is added to the new list `puiseux`.

In [9]:
v=B((4, 4, 5, 6, 7, 3, 4, 5, 2, 1, 3, 2, 4, 3, 6, 7, 5, 4,))
w=B([5, 6, 5, 6, 6, 5, 6, 5, ])
u=L[0]
m=1
u==v*w^m/v

True

In [10]:
puiseux.append([v,w^m,5,2])

We repeat the process.

In [11]:
print(pares0[1])

((1, 4, 5, 6, 4, 5, 3, 4, -3, 5, 2, 3, 4, 7, 6, 7, 1, 5), (2, 4, 6, 3, 4, 2, 5, 7, 6, 7, 4, -6, -7, -6, -5, -6, -5, 3, -5, 4, -3, 5, -4, -2, -5, -6, 5, -3, -4))


In [12]:
v=B((-6, -6, 5, 4, 3, 2,))
w=B([1, ])
u=L[1]
m=1
u==v*w^m/v

True

In [13]:
puiseux.append([v,w^m,1,2])

In [14]:
print(pares0[2])

((1, 4, 5, 6, 4, 5, 3, 4, -3, 5), (2, 3, 4, 7, 6, 7, 5, 4, 3, -5, 6, -4, 5, -3, -4, -6, -5, -6, -7, -2, -6))


In [15]:
v=B((4, -6, 5,  ))
w=B([6,])
u=L[2]
m=1
u==v*w^m/v

True

In [16]:
puiseux.append([v,w^m,6,2])

In [17]:
print(pares0[3])

((1, 4, 5, 6, 4, 5, 3, 4), (-3, 5, 6, 2, 7, 6, 5, 6, 4, 3, -5, 1, 4, 5, 2, 4, -5, 1, 3, -4, 2, -3, 4, 3, -2, -3, -1, -2, -3, -4, -6, -5, -6, -7, -6, -5))


In [18]:
v=B((-6, 5, 4, 3,  ))
w=B([2,])
u=L[3]
m=2
u==v*w^m/v

True

In [19]:
puiseux.append([v,w^m,2,2])

In [20]:
print(pares0[4])

((4, 5, 6, 4, 5, 3), (4, 5, 6, 7, 6, 5, 6, 7, -6, -5, -4, -6, -7, -6))


In [21]:
v=B((4, ))
w=B([3,])
u=L[4]
m=2
u==v*w^m/v

True

In [22]:
puiseux.append([v,w^m,3,2])

In [23]:
print(pares0[5])

((4, 5), (6, 4, 5, 6, 7, 6, 5, -6, -7, -5, -6, -5, -4, 7))


In [24]:
v=B(( ))
w=B([7])
u=L[5]
m=2
u==v*w^m/v

True

In [25]:
puiseux.append([v,w^m,7,2])

In [26]:
print(pares0[6])

((), (4, -7, 5, 4, 5, 6, 5, 7, 3, 6, 1, -5, 4, 5, 6, -7, 4, 3, 2, 1, 3, 2, -3, -4, -6, -5, -6, 7, 6, 5, 6, 7, 6, 5, 4, -6, 3, -2, -3, -1, -2, -3, -4, -6, -5, -4, 5, -6, -3, -7, -5, -6, -5, -4, 7, -5, -4, -1))


In [27]:
v=B((4, 4, 5, 6, 7, 3, 4, 5, 6, -2, 3, -5, 4, -5))
w=B([ 6,])
u=L[6]
m=2
u==v*w^m/v

True

In [28]:
puiseux.append([v,w^m,6,2])

The following braid correspond to a vertical lines having two non-transversal points. It is the product of two commuting algebraic braids and we separate them.

In [29]:
print(pares0[7])

((), (4, -7, 5, 4, 5, 6, 5, 7, 3, 6, 1, -5, 4, 5, 6, 4, 3, 2, 1, 3, 2, 6, 5, 1, 6, 5, 1, 2, 1, -3, -2, 5, 3, -4, -3, -7, -5, -6, -5, -4, 7, -5, -4, -1))


In [30]:
v1=B((4, 4, 5, 6, 7, 4, -6, 5, 3, 4))
w1=B([  6, 5, 6,  6, 5, 6, 5, 5, ])
v2=B((-7, 5, 4, 6, 5, 3, 4))
w2=B([  3, 2, 3,  3, 2, 3, 3, 3, ])
u=L[7]
m1=1
m2=1
u==v1*w1^m1/v1*v2*w2^m2/v2

True

In [31]:
puiseux.append([v1,w1^m1,5,3])
puiseux.append([v2,w2^m2,2,3])

In [32]:
print(pares0[8])

((), (4, -7, 5, 4, 5, 6, 5, 3, 4, -3, -5, 1, 2, 3, 4, -3, -2, 5, -4, -3, 4, -5, -6, -5, -4, 7, -5, -4, -1))


In [33]:
v=B((4,  4, -3, 1))
w=B([ 2])
u=L[8]
m=1
u==v*w^m/v

True

In [34]:
puiseux.append([v,w^m,2,2])

In [35]:
print(pares0[9])

((), (4, -7, 5, 4, 5, 6, 5, -4, 3, 7, 4, -5, 1, 2, 3, -4, -1, -2, -5, -3, -6, -5, -4, -5, -6, -7, -6, 4, 3, 5, 4, 6, 5, 4, 7, 6, 5, 4, -6, -5, -4, -3, 4, 3, 5, 2, 1, 4, -3, -2, 5, -4, -3, 4, -5, -6, -5, -4, 7, -5, -4, -1))


In [36]:
v=B((4, -7, 4, 5,  6,))
w=B([  7, ])
u=L[9]
m=2
u==v*w^m/v

True

In [37]:
puiseux.append([v,w^m,7,2])

In [38]:
print(pares0[10])

((4,), (-7, 5, 4, 5, 6, 5, -4, 3, 4, -5, 1, 2, 3, -4, -1, -2, -1, -5, -3, -4, 3, 4, 3, 2, 5, -3, 4, -5, -6, -5, -4, 7, -5))


In [39]:
v=B((-7, -6, -6, 5,  4, 3, 1))
w=B([  2,])
u=L[10]
m=1
u==v*w^m/v

True

In [40]:
puiseux.append([v,w^m,2,2])

In [41]:
print(pares0[11])

((4,), (-7, 5, 4, 5, 6, 5, -4, 3, -2, -3, 6, -5, 2, 3, -6, 2, -5, 6, 5, 4, -5, -6, -5, -4, 7, -5))


In [42]:
v=B((-7, -6, 5, 4, ))
w=B([ 3,  ])
u=L[11]
m=2
u==v*w^m/v

True

In [43]:
puiseux.append([v,w^m,3,2])

In [44]:
print(pares0[12])

((4,), (-7, 5, 4, 5, 6, 5, -4, -5, -6, 5, -2, 6, -3, 5, -4, 3, 4, 3, -4, -5, 2, -5, 6, 5, 4, -5, -6, -5, -4, 7, -5))


In [45]:
v=B((4, -7, 4, 4, -6, 5,  ))
w=B([ 6,   ])
u=L[12]
m=1
u==v*w^m/v

True

In [46]:
puiseux.append([v,w^m,6,2])

In [47]:
print(pares0[13])

((), (4, -7, 5, 4, 5, 6, 5, -4, -5, -6, 5, 5, 4, 5, 6, 7))


In [48]:
v=B(( -7, -6 ))
w=B([4,  4,  5, 4, 5, 5, 4, 5,  ])
u=L[13]
m=1
u==v*w^m/v

True

In [49]:
puiseux.append([v,w^m,4,3])

The following cell shows the fifteen braids of the braid monodromy. We give to braids $\alpha,\beta$ and the braid is $\alpha\cdot\beta\cdot\alpha^{-1}$.

In [50]:
m=1
for i,j,k,l in puiseux:
    print(m)
    show(i)
    show(j)
    m+=1

1


s3^2*s4*s5*s6*s2*s3*s4*s1*s0*s2*s1*s3*s2*s5*s6*s4*s3

s4*(s5*s4*s5)^2*s4

2


s5^-2*s4*s3*s2*s1

...

s3*s6^-1*s3^2*s5^-1*s4

s5

15


s6^-1*s5^-1

s3^2*(s4*s3*s4)^2

We have all the Puiseux decompositions of the braids. We need to identify to which component each meridian belongs. We see that the generators $\mu_1,\mu_4,\mu_6$ correspond to the cubic. The other ones (and the meridian at infinity) correspond to the lines. We may distinguish the two triangles but we will do it later. This is done looking at the orbits by the monodromy action.

In [51]:
permutaciones=PermutationGroup([_.permutation() for _ in L])
permutaciones.orbits()

[[1, 4, 6], [2], [3], [5], [7], [8]]

We start with the free group corresponding to the fundamental group of the complement of the curve in a vertical line. We add the orbifold relations: the cubes of the meridians of the lines and the 9th power of the meridian of the cubic.

In [52]:
FL8=FreeGroup(8)
L8=[FL8(_)^3 for _ in [[2], [3], [5], [7], [8],[1..8]]]+[FL8([1])^9]

We add the relations of the braid monodromy using the Puiseux decomposition.

In [53]:
for tau1,tau2,m0,m1 in puiseux:
    for j in [m0..m0+m1-1]:
        v=((FL8([j])*tau2)/FL8([j]))*tau1^-1
        L8.append(v)

In [54]:
G=FL8/L8

We simplify the presentation and we keep track of the meridians. We see that this orbifold group is finite. As it is bigger than the abelianization, the group is not abelian.

In [55]:
hom1=G.simplification_isomorphism()
G1=hom1.codomain()

In [56]:
G1.order()

6561

In [57]:
ab=G1.abelian_invariants()
print(ab)
prod(ab)

(3, 3, 3, 3, 3, 9)


2187

We express the meridians in terms of the generators of $G_1$. We denote by $w$ a meridian of the cubic and we put the commutators of the generators in a list.

In [58]:
meridianos=[hom1(G(_)).Tietze() for _ in [[2],[3],[5],[7],[8],[-8..-1],[1]]]
cnm=[G1(a)*G1(b)/G1(a)/G1(b) for a in meridianos for b in meridianos]
w=G1(meridianos[-1])
print(meridianos)

[(2,), (3,), (4,), (5,), (6,), (-6, -5, 6, -1, -6, -1, -4, -3, -2, -1), (1,)]


We compute the derived subgroup (order $3$), we call $t$ a generator and we show its centrality (next three cells)

In [59]:
H=G1.subgroup(cnm)
H.order()

3

In [60]:
t=w*G1(meridianos[0])/w/G1(meridianos[0])
t.order()

3

In [61]:
control=True
for a in G1.generators():
    control=control and a*t/a/t==G1.one()
control

True

We check that the commutators of the meridians of the lines and the meridian of the cubic are of order $3$ and check which ones are equal to $t$.

In [62]:
for a in meridianos[:-1]:
    a1=G1(a)
    c=w*a1/w/a1
    print(c.order(),c==t)

3 True
3 False
3 True
3 False
3 True
3 False


Let us denote by $x_1,x_2,x_3$ the meridians corresponding to the places $0,4,2$ and by $y_1,y_2,y_3$ the meridians corresponding to the places $1,3,5$. We are going to check the presentation given in Proposition 4.3

In [63]:
x1,x2,x3=[G1(meridianos[j]) for j in (0,4,2)]
y1,y2,y3=[G1(meridianos[j]) for j in (1,3,5)]

In [78]:
control=True
for a,b in [(x1,x2),(x2,x3),(x3,x1)]:
    control=control and (a*b/a/b)==t
control

True

In [79]:
control=True
for a,b in [(y1,y2),(y2,y3),(y3,y1)]:
    control=control and (a*b/a/b)==t
control

True

In [66]:
control=True
for a in [x1,x2,x3]:
    for b in [y1,y2,y3]:
        control=control and (a*b/a/b).order()==1
control

True

In [67]:
control=True
for a in [x1,x2,x3]:
    control=control and w*a/w/a==t
control

True

In [68]:
control=True
for a in [y1,y2,y3]:
    control=control and w*a/w/a==t^-1
control

True

We have proved that $G_1$ has the following properties.
- It fits in a short exact sequence
$$
1\longrightarrow\mathbb{Z}/3\longrightarrow G_1\longrightarrow\left(\mathbb{Z}/3\right)^5\times\mathbb{Z}/9
\longrightarrow 1
$$
where the kernel is $G'_1$ and the extension is central and the cokernel is presented as the abelian group 
generated by $x_1,x_2,x_3,y_1,y_2,y_3,w$, where
$$
x_i^3=y_i^3=w^9=x_1\cdot x_2\cdot x_3\cdot y_1\cdot y_2\cdot y_3\cdot w=1.
$$
- It is generated by $x_1,x_2,x_3,y_1,y_2,y_3,w,t$ with the following relations:
    * $t^3=1$
    * $[t,x_i]=[t,y_i]=[t,w]=1$
    * $x_i^3=y_i^3=w^9=1$
    * $x_1\cdot x_2\cdot x_3\cdot y_1\cdot y_2\cdot y_3\cdot w=t$
    * $[x_1,x_2]=[x_2,x_3]=[x_3,x_1]=[y_1,y_2]=[y_2,y_3]=[y_3,y_1]=t$
    * $[x_i,y_j]=1$
    * $[w,x_i]=[y_j,w]=t$
    
The cells below end the proof.

In [71]:
control
for a in [x1,x2,x3,y1,y2,y3]:
    control=control and a^3==G1.one()
control

True

In [73]:
w^9==G1.one()

True

In [77]:
prod([x1,x2,x3,y1,y2,y3]+3*[w])==t

True